## Key-value базы данных: Redis

Установка:
```bash
sudo apt-get install redis-server
pip3 install redis
```

In [ ]:
import redis

r = redis.Redis(host='localhost', port=6379, db=13)                                                        

In [ ]:
print(r.get('foo'))

In [ ]:
r.set('foo', 'bar')                                                     

In [ ]:
r.get('foo')   

Подробнее: https://pypi.org/project/redis/

## GIL (Global Interpeter Lock) в Python

Начнем сразу с примера программы:

In [ ]:
def count(n):
    while n > 0:
        n -= 1
        
x = 100000000

Запустим два раза и измерим время:

In [ ]:
%%time
count(x)
count(x)

Также запустим две функции, но теперь они будут выполняться параллельно:

In [ ]:
from threading import Thread

In [ ]:
%%time
t1 = Thread(target=count,args=(x,))
t1.start()
t2 = Thread(target=count,args=(x,))
t2.start()
t1.join()
t2.join()

В любой момент может выполняться только один поток Python. Глобальная блокировка интерпретатора — GIL — тщательно контролирует выполнение тредов. GIL гарантирует каждому потоку эксклюзивный доступ к переменным интерпретатора (и соответствующие вызовы C-расширений работают правильно).

Потоки удерживают GIL, пока выполняются. Однако они освобождают его при блокировании для операций ввода-вывода. Каждый раз, когда поток вынужден ждать, другие, готовые к выполнению, потоки используют свой шанс запуститься.

При работе с CPU-зависимыми потоками, которые никогда не производят операции ввода-вывода, интерпретатор периодически проводит проверку («the periodic check»).

По умолчанию это происходит каждые 100 «тиков»(тики неделимые инструкции python), но этот параметр можно изменить с помощью sys.setcheckinterval(). Интервал проверки — глобальный счетчик, абсолютно независимый от порядка переключения потоков. (В новых версиях python, проверка происходит по времени, а не по тикам).

При периодической проверке в главном потоке запускаются обработчики сигналов, если таковые имеются. Затем GIL отключается и включается вновь. На этом этапе обеспечивается возможность переключения нескольких CPU-зависимых потоков (при кратком освобождении GIL другие треды имеют шанс на запуск).

### А зачем нужен GIL? 

Python подсчитывает количество ссылок для корректного управления памятью. Это означает, что созданные в Python объекты имеют переменную подсчёта ссылок, в которой хранится количество всех ссылок на этот объект. Как только эта переменная становится равной нулю, память, выделенная под этот объект, освобождается.

Вот небольшой пример кода, демонстрирующий работу переменных подсчёта ссылок:

In [ ]:
import sys

a = []
b = a

sys.getrefcount(a)

В этом примере количество ссылок на пустой список равно 3. На этот список ссылаются: переменная a, переменная b и аргумент, переданный функции sys.getrefcount().

Проблема, которую решает GIL, связана с тем, что в многопоточном приложении сразу несколько потоков могут увеличивать или уменьшать значения этого счётчика ссылок. Это может привести к тому, что память очистится неправильно и удалится тот объект, на который ещё существует ссылка.

Счётчик ссылок можно защитить, добавив блокираторы на все структуры данных, которые распространяются по нескольким потокам. В таком случае счётчик будет изменяться исключительно последовательно.

Но добавление блокировки к нескольким объектам может привести к появлению другой проблемы — взаимоблокировки (англ. deadlocks), которая получается только если блокировка есть более чем на одном объекте. 

### Все о чем мы говорили выше $-$ относится к потокам, а чем поток отличается от процесса?

In [ ]:
from multiprocessing import Process, Pipe

def f(conn):
    conn.send([42, None, 'hello'])
    print('hello')
    conn.close()

if __name__ == '__main__':
    parent_conn, child_conn = Pipe()
    p = Process(target=f, args=(child_conn,))
    p.start()
    print(parent_conn.recv())   # prints "[42, None, 'hello']"
    p.join()

Если мы используем multiprocessing $-$ таких проблем не будет, но у такого подхода есть и свои минусы. Например, не удобно использвовать ```pipes```, для передачи результатов: [документация](https://docs.python.org/3/library/multiprocessing.html)


## Визуализация графов в Python, библиотека PyGraphviz:

In [3]:
from graphviz import Digraph

g = Digraph('G')

g.edge('Hello', 'World')

g

ModuleNotFoundError: No module named 'graphviz'